In [1]:
import pandas as pd
import datetime as dt
import uuid

In [2]:
from bright_data import load_data

In [3]:
%load_ext autoreload
%autoreload 2

### Build Example `Alert-Objects`

In [4]:
alert_1 = {
    
    'alert_id':         uuid.uuid4(),
    'user_id':          12,
    'alert_set_dt':     dt.datetime(2022, 8, 1),
    'alert_expires_dt': dt.datetime(2022, 10, 1),
    'expires_after':    False,
    'alert_type':       'seats',
    'event_id':         '0E005C56E48E2B02',
    
    'alert_obj': {
        'alert_type': 'availability',
        'section': '204',
        'row':     None,
        'operator': '<',
        'quantity': 6,
        
    }
}

alert_2 = {
    
    'alert_id':         uuid.uuid4(),
    'user_id':          12,
    'alert_set_dt':     dt.datetime(2022, 8, 2),
    'alert_expires_dt': dt.datetime(2022, 10, 1),
    'expires_after':    False,
    'alert_type':       'seats',
    'event_id':         '0E005C56E48E2B02',
    
    'alert_obj': {
        'alert_type': 'availability',
        'section': '203',
        'row':     None,
        'operator': '<',
        'quantity': 2,
        
        
    }
}

alert_3 = {
    
    'alert_id':         uuid.uuid4(),
    'user_id':          6,
    'alert_set_dt':     dt.datetime(2022, 8, 5),
    'alert_expires_dt': dt.datetime(2022, 10, 1),
    'expires_after':    True,
    'alert_type':       'seats',
    'event_id':         '0E005C56E48E2B02',
    
    'alert_obj': {
        'alert_type': 'price',
        'section': '203',
        'operator': '<',
        'quantity': 60.0,
        
    }
}

### Skeleton of alert filtering system

In [5]:
def constraint_check(amt, operator, quantity):
    '''
        returns True if the amt/operator/quantity satisfy 
        a particular relationship; otherwise False
    '''
    
    if operator == '<':
        return (amt < quantity)
    elif operator == '=':
        return (amt == quantity)
    elif operator == '>':
        return (amt > quantity)
    else:
        return False

    
def inner_check(alert_obj, data_old, data_new):
    '''
        returns True if the alert should be triggered, False otherwise
        Need to check the `alert_type` for the column of interest
        and the `section` field if we should break it into sections
    '''
    
    if alert_obj['alert_type'] == 'availability':
        
        if alert_obj['section'] is not None:
            tbl = data_new.groupby('section').agg('sum')['availability']
            amt = tbl[alert_obj['section']]
        else:
            amt = data_new.agg('sum')['availability']
        
    
    elif alert_obj['alert_type'] == 'price':
        
        if alert_obj['section'] is not None:
            tbl = data_new.groupby('section').agg('min')['price']
            amt = tbl[alert_obj['section']]
        else:
            amt = data_new.agg('min')['price']
    
    else:
        pass
    
    # now check if the contraint is true or false
    result = constraint_check(
                amt,
                alert_obj['operator'],
                alert_obj['quantity'],
            )
        
    return result
    

def outer_check(event_id, alert, data_old, data_new):
    
    # check if valid for datetime, alert_type
    if (
        (alert['alert_set_dt'] > dt.datetime.now())    or
        (alert['alert_expires_dt'] < dt.datetime.now())
    ):
        # TODO - we can remove expired alerts here
        return 
        

    if alert['alert_type'] != 'seats':
        # TODO - there are other type of alerts like those
        # based on changes to the scheduling of the event,
        # but those don't use bright_data scrapes so we don't 
        # use them here
        return 
    
    # check if the alert should be triggered
    try:
        result = inner_check(
                    alert['alert_obj'],
                    data_old,
                    data_new,
        )
    except:
        result = False
        
    if result:
        triggered_obj = {
            'alert_id':   alert['alert_id'],
            'user_id':    alert['user_id'],
            'trigger_dt': dt.datetime.now(),
            'alert':      alert,
            'trigger_amt': None   # TODO - we might want to add the amt that 
            # triggered the alert, e.g. if it was looking for tickets below
            # $50 and found one for $40 this value should be 40.
        }
        
        alerts_triggered.append(triggered_obj)
        
        if alert['expires_after']:
            
            ind = [i for (i,v) in enumerate(alerts_listening) 
                   if alert['alert_id']
                  ][0]
            alerts_listening.pop(ind)
            
            



### Read in Data

In [13]:
data_old = load_data('bright_data_1-9.1.22.json')
data_new = load_data('bright_data_1-9.2.22.json')

### Demo the Alert system

##### These arrays would be DB tables

In [14]:
alerts_listening = [alert_1, alert_2, alert_3]
alerts_triggered = []

##### Parse all the alerts_listening to just the `type` and for the `event_id` which we received new data on

In [15]:
event_id = "0E005C56E48E2B02"
alert_type = "seats"

alerts = [alert for alert in alerts_listening
          if (
              (alert['event_id'] == event_id)    and
              (alert['alert_type'] == alert_type)
             )
         ]

In [16]:
data_old = data_old[data_old['event_id'] == event_id]
data_new = data_new[data_new['event_id'] == event_id]

##### It all happens right here!

In [17]:
for alert in alerts:
    outer_check(event_id, alert, data_old, data_new)

### Results

We start with 3 listening and 0 tirggered and end with 2 listening and 2 triggered.

This is because alert_1 and alert_3 are setup to trigger, and then alert_3 also has `expire_after=True` so that when it's triggered, it leaves the listening pool.

In [11]:
len(alerts_listening), len(alerts_triggered)

(2, 2)

In [12]:
alerts_triggered

[{'alert_id': UUID('bd43f926-14cc-4224-8756-9566056382ee'),
  'user_id': 12,
  'trigger_dt': datetime.datetime(2022, 8, 12, 14, 11, 51, 604302),
  'alert': {'alert_id': UUID('bd43f926-14cc-4224-8756-9566056382ee'),
   'user_id': 12,
   'alert_set_dt': datetime.datetime(2022, 8, 1, 0, 0),
   'alert_expires_dt': datetime.datetime(2022, 10, 1, 0, 0),
   'expires_after': False,
   'alert_type': 'seats',
   'event_id': '0E005C56E48E2B02',
   'alert_obj': {'alert_type': 'availability',
    'section': '204',
    'row': None,
    'operator': '<',
    'quantity': 6}},
  'trigger_amt': None},
 {'alert_id': UUID('067ebb26-4ac5-4ed0-a0f9-410866284a4e'),
  'user_id': 6,
  'trigger_dt': datetime.datetime(2022, 8, 12, 14, 11, 51, 660025),
  'alert': {'alert_id': UUID('067ebb26-4ac5-4ed0-a0f9-410866284a4e'),
   'user_id': 6,
   'alert_set_dt': datetime.datetime(2022, 8, 5, 0, 0),
   'alert_expires_dt': datetime.datetime(2022, 10, 1, 0, 0),
   'expires_after': True,
   'alert_type': 'seats',
   'event_